In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc.filter.matchedfilter import sigma
from calcwf import *

/home/ben.patterson/.conda/envs/igwn_eccentric/lib/python3.10/site-packages/pycbc/types/array.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal as _lal


lal.MSUN_SI != Msun
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new


So far we have been using a lower frequency in all match functions of f_low+3, so virtually always 13 Hz. The problem is that generating harmonics at 10Hz causes h1 to start at (just under) 15 Hz, and h2 at (just under) 20 Hz. This is therefore not quite correct. In reality current detectors struggle to see anything below 20 Hz and so our current psd of aLIGOZeroDetHighPower is somewhat optimistic in this regard. Nevertheless we will calculate the power loss with this psd of increasing the lower frequency cutoff from 13 to 20 Hz.

We will assume effect when calculating harmonics themselves to be negligible and continue using 13Hz there for now until/if we change it globally.

In [2]:
_, h0, h1, hn1, h2, _, _, _, _ = get_h([1,1,1,1], 10, 0.3, chirp2total(24,2), 2, 4096)
psd = gen_psd(h0, 10)
for harm in [h0, h1, hn1, h2]:
    harm_f = harm.real().to_frequencyseries()
    remaining_power = sigma(harm_f, psd, low_frequency_cutoff=20, high_frequency_cutoff=psd.sample_frequencies[-1])
    total_power = sigma(harm_f, psd, low_frequency_cutoff=13, high_frequency_cutoff=psd.sample_frequencies[-1])
    print(f'{(total_power-remaining_power)*100/total_power:.2f}% loss in power')

1.14% loss in power
3.94% loss in power
2.21% loss in power
1.71% loss in power


No matter the choice of eccentricity or chirp mass (or mass ratio), there is never more than about a 5% power loss in any harmonic, and usually less than this. This seems acceptable, and will again be more realistic anyway with regards to how far low in frequency we can actually see.

Now changed globally. By default get_h() etc. will use 20Hz however can also override in (almost) all relevant functions with f_match parameter.